In [30]:
import pandas as pd
import plotly.express as px
import re

In [2]:
df = pd.read_csv('crypto_data.csv')

In [91]:
def plot_percent_change_agg(data, aggregation_type, coin=None):
    """Generates a line plot to explore the percent change of coins
    
    Args:
        data (pd.DataFrame): crypto data as pandas dataframe
        aggregation_type (str): pandas aggregation method ('mean','median')
        coin (list of str or str, optional): specified crypto coin to include in plot
    """
    
    data = data.copy()
    
    # get specific coin data
    if coin:
        # single coin
        if type(coin) == str:
            data = data[data.name.str.lower() == coin.lower()]
        # input is list for single or multiple coins
        else:
            coin = [name.lower() for name in coin]
            data = data[data.name.str.lower().isin(coin)]

    # percent change columns
    cols = [col for col in data.columns if 'percent_change' in col]

    # group coins and aggregate by specified input
    plot_data = data.groupby('name', sort=False)[cols].agg(aggregation_type)

    # stack and rename columns
    plot_data = plot_data.stack().reset_index()
    plot_data.columns = ['name','period','value']

    # extract only relevant period
    plot_data.period = plot_data.period.str.extract('percent_change_(.+)')

    fig = px.line(plot_data, x='period', y='value', color='name', markers=True, title=f'Crypto {aggregation_type.title()} Percent Change')
    fig.update_layout(
        xaxis_title='Timestamp',
        yaxis_title='Percent Change'
    )

    fig.show()

In [ ]:
def plot_coin_line(data, coin, y_column_name):
    """Generates a line plot for both a specified coin(s) and column name, such as 'quote.AUD.price'

    Args:
        data (pd.DataFrame): crypto data 
        coin (list of str or str): specified crypto coin(s)
        y_column_name (str): specified column name

    """
    
    if type(coin) == str:
        plot_data = data[['name',y_column_name,'timestamp']][data.name.str.lower() == coin.lower()]
    else:
        coin = [name.lower() for name in coin]
        plot_data = data[['name',y_column_name,'timestamp']][data.name.str.lower().isin(coin)]

    # column name without 'quote.CURRENCY.'
    col_name = re.findall('quote\..+\.(.+)', y_column_name)[0]

    fig = px.line(plot_data, x='timestamp', y=y_column_name, color='name', markers=True, title=f'Crypto {col_name.title()} over time')
    fig.update_layout(
        xaxis_title='Timestamp',
        yaxis_title=col_name
    )

    fig.show()
    

In [92]:
# mean
plot_percent_change_agg(df, 'mean', coin=['Bitcoin','Ethereum'])

In [55]:
# price
plot_coin_line(df, coin=['bitcoin','ethereum'], y_column_name='quote.AUD.price')

In [57]:
# marketcap
plot_coin_line(df, coin=['bitcoin','ethereum'], y_column_name='quote.AUD.market_cap')

In [54]:
# volume 24h change
plot_coin_line(df, coin=['bitcoin','ethereum'], y_column_name='quote.AUD.volume_change_24h')

# volume 24h
plot_coin_line(df, coin=['bitcoin','ethereum'], y_column_name='quote.AUD.volume_24h')
